In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import os 
print(os.listdir('/kaggle/input/eurosat-dataset/EuroSAT/'))
path = '/kaggle/input/eurosat-dataset/EuroSAT/'

import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory
from matplotlib import image
from PIL import Image

path = '/kaggle/input/eurosat-dataset/EuroSAT/'

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [ ]:
dataset = image_dataset_from_directory('/kaggle/input/eurosat-dataset/EuroSAT',labels = "inferred", label_mode = 'categorical', image_size = (64, 64))

In [ ]:
path = '/kaggle/input/eurosat-dataset/EuroSAT/'

sample1df = pd.read_csv(path + 'train.csv')
sample2df = pd.read_csv(path + 'validation.csv')
sample3df = pd.read_csv(path + 'test.csv')

s1fname = sample1df.Filename
s2fname = sample2df.Filename
s3fname = sample3df.Filename

s1label = sample1df.Label
s2label = sample2df.Label
s3label = sample3df.Label

In [ ]:
image1 = []
for fname in s1fname:
    Image = image.load_img(path + fname)  
    image_arr = image.img_to_array(Image)
    image1.append(image_arr)

In [ ]:
image2 = []
for fname in s2fname:
    Image = image.load_img(path + fname)  
    image_arr = image.img_to_array(Image)
    image2.append(image_arr)


In [ ]:
image3 = []
for fname in s3fname:
    Image = image.load_img(path + fname)  
    image_arr = image.img_to_array(Image)
    image3.append(image_arr)

In [ ]:
#data 정리 
x = image1 + image2
y = np.array(list(s1label) + list(s2label))
xtest = image3
ytest = np.array(s3label)

#normalization 
x = np.array(x) / 255.
xtest = np.array(xtest) / 255.

#one-hot encoding 
y = to_categorical(y, 10)
ytest= to_categorical(ytest, 10)

print(x.shape)
print(y.shape)
print(xtest.shape)
print(ytest.shape)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(x, y, test_size = 0.22, shuffle = True, stratify = y, random_state = 3)

In [ ]:
print(xtrain.shape)
print(ytrain.shape)
print(xval.shape)
print(yval.shape)

In [ ]:
np.random.seed(3)

augGen = ImageDataGenerator(rotation_range = 15, width_shift_range = 0.2, height_shift_range = 0.2)

xtrain_generator = augGen.flow(xtrain, ytrain, batch_size = 32, save_format = 'jpeg')

# Transfer Learning 
## 1) ResNet50

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow import keras

In [ ]:
resnet50 = ResNet50(weights = 'imagenet', include_top = False, input_shape = (64, 64, 3))

In [ ]:
print(len(resnet50.layers))

In [ ]:
resnet50.summary() # top 만 freeze

In [ ]:
fine_tune_at = len(resnet50.layers) - 5

for layer in resnet50.layers[:fine_tune_at]:
    layer.trainable = False

# top을 포함한 5개 제외하고 나머지 layer들 모두 freeze

In [ ]:
resnet50.summary()

In [ ]:
x = resnet50.output

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, Activation 
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.2)(x)
x = Dense(10, activation = 'softmax')(x)

In [ ]:
from tensorflow.keras.models import Model
resnet50_model = Model(inputs = resnet50.input, outputs = x)

In [ ]:
resnet50_model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
import os
checkpoint_path = "resnet__train/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True, period = 1)

In [ ]:
resnet50_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = resnet50_model.fit(xtrain, ytrain, validation_data = (xval, yval), epochs = 20, callbacks = [cp_callback])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

epochs = range(1, len(loss) + 1)

plt.figure()
plt.plot(epochs, accuracy, 'r', label = 'accuracy')
plt.plot(epochs, val_accuracy,'b', label = 'val_accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'r', label = 'loss')
plt.plot(epochs, val_loss, 'b', label = 'val_loss')
plt.legend()